In [ ]:
# Установка необходимых библиотек
!pip install -q transformers==4.36.0
!pip install -q peft==0.7.1
!pip install -q trl==0.7.4
!pip install -q bitsandbytes==0.41.3
!pip install -q accelerate==0.25.0
!pip install -q datasets==2.15.0

print("✅ Библиотеки установлены")


In [ ]:
# Импорты
import json
import torch
import pandas as pd
from pathlib import Path
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import Dataset
import shutil
import os

print(f"🔥 CUDA доступен: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(
        f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB"
    )

In [ ]:
# Загрузка конфигурации
# Замените '/kaggle/input/your-dataset' на путь к вашему dataset в Kaggle
DATASET_PATH = "/kaggle/input/hashtag-dataset"  # Измените на ваш dataset

config_file = Path(DATASET_PATH) / "config.json"
with open(config_file, "r", encoding="utf-8") as f:
    config = json.load(f)

print("📋 Конфигурация загружена:")
print(f"   Модель: {config['model_config']['base_model']}")
print(f"   Обучающих примеров: {config['dataset_info']['train_samples']}")
print(f"   Тестовых примеров: {config['dataset_info']['test_samples']}")
print(f"   LoRA rank: {config['training_config']['lora_r']}")

In [ ]:
# Загрузка данных
def load_jsonl(file_path):
    """Загружает данные из JSONL файла"""
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line))
    return data


# Загружаем обучающие и тестовые данные
train_file = Path(DATASET_PATH) / "train_data.jsonl"
test_file = Path(DATASET_PATH) / "test_data.jsonl"

train_data = load_jsonl(train_file)
test_data = load_jsonl(test_file)

print(f"📊 Загружено:")
print(f"   Обучающих: {len(train_data)}")
print(f"   Тестовых: {len(test_data)}")

# Пример данных
print(f"\n📝 Пример обучающего примера:")
print(f"Вход: {train_data[0]['input'][:100]}...")
print(f"Выход: {train_data[0]['output']}")

In [ ]:
# Создание папок для результатов (создаются в Kaggle)
output_dir = "./hashtag_lora_model"
merged_dir = "./merged_hashtag_model"
final_dir = "./final_model_for_ollama"

os.makedirs(output_dir, exist_ok=True)
os.makedirs(merged_dir, exist_ok=True)
os.makedirs(final_dir, exist_ok=True)

print(f"📁 Папки созданы в Kaggle:")
print(f"   LoRA модель: {output_dir}")
print(f"   Объединенная модель: {merged_dir}")
print(f"   Финальная модель: {final_dir}")

In [ ]:
# Полный код обучения (объединен для упрощения)
# Настройка квантизации
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Загрузка модели
model_name = config["model_config"]["base_model"]
print(f"🔄 Загрузка модели: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model = prepare_model_for_kbit_training(model)

# Настройка LoRA
lora_config = LoraConfig(
    r=config["training_config"]["lora_r"],
    lora_alpha=config["training_config"]["lora_alpha"],
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("✅ Модель и LoRA настроены")

In [ ]:
# Подготовка данных и запуск обучения
def format_dataset(examples):
    """Форматирует данные для обучения"""
    return [example["text"] for example in examples]


# Создаем датасеты
train_texts = format_dataset(train_data)
test_texts = format_dataset(test_data)

train_dataset = Dataset.from_dict({"text": train_texts})
test_dataset = Dataset.from_dict({"text": test_texts})

# Настройка обучения
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=config["training_config"]["epochs"],
    per_device_train_batch_size=config["training_config"]["batch_size"],
    per_device_eval_batch_size=config["training_config"]["batch_size"],
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=10,
    learning_rate=config["training_config"]["learning_rate"],
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to=None,
    evaluation_strategy="steps",
    eval_steps=50,
)

# Создание тренера
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_args,
    max_seq_length=512,
    packing=False,
)

print("🚀 Начинаем обучение...")
trainer.train()
print("🎉 Обучение завершено!")

In [ ]:
# Сохранение и подготовка финальной модели
# Сохранение LoRA
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Объединение LoRA с базовой моделью
print("🔄 Объединение LoRA с базовой моделью...")

# Загружаем базовую модель без квантизации
base_model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)

# Загружаем LoRA и объединяем
from peft import PeftModel

model_with_lora = PeftModel.from_pretrained(base_model, output_dir)
merged_model = model_with_lora.merge_and_unload()

# Сохраняем объединенную модель
merged_model.save_pretrained(merged_dir)
tokenizer.save_pretrained(merged_dir)

print(f"✅ Объединенная модель сохранена в {merged_dir}")

In [ ]:
# Создание финальной модели для Ollama
print("📦 Подготовка модели для Ollama...")

# Копируем объединенную модель в финальную папку
shutil.copytree(merged_dir, final_dir, dirs_exist_ok=True)

# Создаем Modelfile для Ollama
modelfile_content = '''FROM ./

SYSTEM """Ты эксперт по генерации хештегов для новостей, специально обученный на российских новостных данных.

Твоя задача - анализировать новостные тексты и генерировать релевантные хештеги на русском языке.

Правила генерации:
1. Создавай 3-5 хештегов для каждой новости
2. Хештеги должны быть конкретными и описательными
3. Используй только русский язык
4. Не добавляй символ # в ответе
5. Разделяй хештеги запятыми
6. Фокусируйся на ключевых темах и событиях

Отвечай только хештегами, без дополнительных комментариев."""

PARAMETER temperature 0.1
PARAMETER top_p 0.9
PARAMETER repeat_penalty 1.1
PARAMETER num_ctx 2048
PARAMETER stop "<s>"
PARAMETER stop "</s>"
'''

with open(f"{final_dir}/Modelfile", "w", encoding="utf-8") as f:
    f.write(modelfile_content)

# Создаем инструкцию
instructions = f"""# 🚀 Дообученная модель Saiga LLaMA3 для генерации хештегов

## 📊 Информация об обучении
- Базовая модель: {model_name}
- Обучающих примеров: {config['dataset_info']['train_samples']}
- Эпох обучения: {config['training_config']['epochs']}
- LoRA rank: {config['training_config']['lora_r']}

## 🔧 Установка в Ollama

1. Скачайте и распакуйте эту папку
2. Перейдите в папку с моделью:
   ```bash
   cd path/to/final_model_for_ollama
   ```

3. Создайте модель в Ollama:
   ```bash
   ollama create saiga-hashtag-pro -f Modelfile
   ```

4. Проверьте установку:
   ```bash
   ollama list
   ```

## 🧪 Тестирование

```bash
ollama run saiga-hashtag-pro "Центральный банк повысил ключевую ставку"
```
"""

with open(f"{final_dir}/README.md", "w", encoding="utf-8") as f:
    f.write(instructions)

print(f"✅ Финальная модель подготовлена в {final_dir}")

In [ ]:
# Создание архива для скачивания
print("📦 Создание архива для скачивания...")

shutil.make_archive("saiga_hashtag_model_for_ollama", "zip", final_dir)

# Информация о размерах
archive_size = os.path.getsize("saiga_hashtag_model_for_ollama.zip") / (1024**3)
print(f"✅ Архив создан: saiga_hashtag_model_for_ollama.zip")
print(f"📊 Размер архива: {archive_size:.2f} GB")

print("\n🎉 ГОТОВО!")
print("📥 Скачайте файл 'saiga_hashtag_model_for_ollama.zip'")
print("🚀 Следуйте инструкциям в README.md внутри архива")
print("\n📋 Следующие шаги:")
print("1. Скачайте архив из Kaggle")
print("2. Распакуйте на локальной машине")
print("3. Выполните команду: ollama create saiga-hashtag-pro -f Modelfile")
print("4. Интегрируйте в ваш проект согласно KAGGLE_TO_OLLAMA_GUIDE.md")